### Imports
This cell imports all the necessary libraries. We'll need json to load our data and a lot of components from qiskit and qiskit_optimization.

In [ ]:
#  Cell 1: Imports 

import json
from qiskit_algorithms import VQE,QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_optimization.applications import OptimizationApplication
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit.circuit.library import QAOAAnsatz
from qiskit_algorithms import SamplingVQE
from qiskit.circuit.library import QAOAAnsatz
from qiskit import transpile
from qiskit.circuit.library import RealAmplitudes
print("Libraries imported successfully.")

Libraries imported successfully.


Load Problem Data
This cell loads the problem_data.json file you just created.

In [ ]:
#  Cell 2: Load Problem Data 

with open('problem_data.json', 'r') as f:
    data = json.load(f)

#  Extract data for easier access 
tails = data['tails']
flights = data['flights']
costs = data['costs']
max_costs = data['max_costs']
impossible_pairings = data['impossible_pairings']
params = data['tuning_parameters']

#  Create quick-access lists of IDs 
tail_ids = [t['id'] for t in tails]
flight_ids = [f['id'] for f in flights]

print(f"Loaded {len(flights)} flights and {len(tails)} tails.")
print(f"Tuning Parameters: {params}")

Loaded 5 flights and 4 tails.
Tuning Parameters: {'gamma': 5.0, 'eta': 8.5, 'lambda': 3.0, 'tau': 4.0, 'varphi': 4.0, 'psi': 0.3}


Initialize Quadratic Program & Define Variables
Here, we create the QuadraticProgram object and add our 40 binary variables, one for each (flight, tail) pair. We'll name them q_F1_A101, q_F1_B202, etc.

In [ ]:
#  Cell 3: Initialize Quadratic Program & Define Variables 

# Initialize the Quadratic Program
qp = QuadraticProgram(name="Tail-Assignment-Problem")

# Add all 40 binary variables (8 flights * 5 tails)
# The name format "q_{f}_{t}" will be a useful key
for f in flight_ids:
    for t in tail_ids:
        var_name = f"q_{f}_{t}"
        qp.binary_var(name=var_name)

print(f"Added {len(qp.variables)} binary variables to the Quadratic Program.")
print("First 5 variables:", [var.name for var in qp.variables[:5]])

Added 20 binary variables to the Quadratic Program.
First 5 variables: ['q_F1_A101', 'q_F1_B202', 'q_F1_C303', 'q_F1_D404', 'q_F2_A101']


Define the Objective Function (H_C)This cell builds the objective function $H_C$. We loop through our costs data and add each term to the QuadraticProgram.$$H_C = \sum_{f \in F} \sum_{t \in T_f} \left( \frac{\text{execcost}_{f,t}}{\text{maxcost}_f} \right) q_{f,t}$$

In [ ]:
#  Cell 4: Define the Objective Function (H_C) 

# The objective is linear, so we build a dictionary of linear terms
# {variable_name: coefficient}
objective_linear_terms = {}

for f, tail_costs in costs.items():
    for t, exec_cost in tail_costs.items():
        var_name = f"q_{f}_{t}"
        
        # Calculate normalized cost
        normalized_cost = exec_cost / max_costs[f]
        
        # Add the term: (psi * normalized_cost) * q_{f}_{t}
        objective_linear_terms[var_name] = params['psi'] * normalized_cost

# Set the objective function in the Quadratic Program
qp.minimize(linear=objective_linear_terms)

print("Objective function (H_C) defined and set.")

Objective function (H_C) defined and set.


Add Assignment Constraint (H_A)This is the "one-hot" constraint: $\sum_{t} q_{f,t} = 1$ for each flight $f$.Qiskit's qp.linear_constraint makes this easy.$$H_A = \sum_{f \in F} \left( \sum_{t \in T} q_{f,t} - 1 \right)^2$$

In [ ]:
#  Cell 5: Add Assignment Constraint (H_A) 

# H_A: Each flight must be assigned to exactly one tail
# We add one constraint for each flight

for f in flight_ids:
    # Create a dictionary of variables for this constraint: {var_name: 1}
    linear_vars = {f"q_{f}_{t}": 1 for t in tail_ids}
    
    # Add the constraint: sum(vars) == 1
    # We give it a name 'H_A_F1', 'H_A_F2', etc.
    qp.linear_constraint(
        linear=linear_vars,
        sense="==",
        rhs=1,
        name=f"H_A_{f}"
    )

print(f"Added {len(flight_ids)} 'H_A' constraints (one per flight).")

Added 5 'H_A' constraints (one per flight).


Add Aircraft Constraint (H_B4)This constraint penalizes any assignment of a flight $f$ to a tail $t$ that doesn't meet the fleet or seat requirements. This is a simple linear penalty.$$H_{B4} = \sum_{(f,t) \in \text{Invalid}} q_{f,t}$$

In [ ]:
#  Cell 5b: Add Aircraft Constraint (H_B4) 

# H_B4: Penalize invalid assignments (wrong fleet or not enough seats)
# This is a linear penalty, so we add to the objective.

linear_penalty_B4 = {}
all_tails_map = {t['id']: t for t in tails}

for f in flights:
    for t_id in tail_ids:
        tail = all_tails_map[t_id]
        var_name = f"q_{f['id']}_{t_id}"

        # Check for incompatibility
        is_invalid = False
        if f['fleet_req'] != tail['fleet']:
            is_invalid = True
        if f['seats_req'] > tail['capacity']:
            is_invalid = True
            
        # If it's an invalid assignment, add the penalty 'varphi'
        if is_invalid:
            linear_penalty_B4[var_name] = params['varphi']

# Add these linear penalties to the existing objective
qp.minimize(linear=linear_penalty_B4)

print(f"Added linear penalties (H_B4) for {len(linear_penalty_B4)} invalid assignments.")

Added linear penalties (H_B4) for 13 invalid assignments.


Add Maintenance Constraint (H_B3)This constraint penalizes any assignment of a flight $f$ to a tail $t$ if that flight occurs on the tail's required maintenance day.$$H_{B3} = \sum_{t \in T_{\text{main}}} \sum_{f \in F_{\text{day}, t}} q_{f,t}$$

In [ ]:
# Cell 5c: Add Maintenance Constraint (H_B3) 

# H_B3: Penalize assignments on a tail's maintenance day.
# This is also a linear penalty added to the objective.

linear_penalty_B3 = {}

for t in tails:
    # Check if this tail requires maintenance
    if t['maintenance']:
        maint_day = t['maintenance']['day']
        
        # Find all flights on that day
        for f in flights:
            if f['departure']['day'] == maint_day:
                # This flight is on the maintenance day
                var_name = f"q_{f['id']}_{t['id']}"
                
                # Add the penalty 'tau'
                linear_penalty_B3[var_name] = params['tau']

# Add these linear penalties to the existing objective
qp.minimize(linear=linear_penalty_B3)

print(f"Added linear penalties (H_B3) for {len(linear_penalty_B3)} maintenance conflicts.")

Added linear penalties (H_B3) for 0 maintenance conflicts.


Add Impossible Pairings Constraint (H_B1)This constraint penalizes any solution where two operationally impossible flights (like F1 and F2) are assigned to the same tail. This is a quadratic penalty.$$H_{B1} = \sum_{t \in T} \sum_{(f, f') \in I_f} q_{f,t} \cdot q_{f',t}$$

In [ ]:
#  Cell 5d: Add Impossible Pairings Constraint (H_B1) 

# H_B1: Penalize assigning impossible flight pairs to the *same tail*.
# This is a quadratic penalty: eta * q_{f,t} * q_{f',t}

quadratic_penalty_B1 = {}

for t_id in tail_ids:
    # For each pair of flights (f1, f2) that are impossible to pair
    for f1, f2 in impossible_pairings:
        
        var1_name = f"q_{f1}_{t_id}"
        var2_name = f"q_{f2}_{t_id}"
        
        # Add the quadratic penalty term
        # Qiskit handles (var1, var2) or (var2, var1) the same
        quadratic_penalty_B1[(var1_name, var2_name)] = params['eta']

# Add these quadratic penalties to the existing objective
qp.minimize(quadratic=quadratic_penalty_B1)

print(f"Added quadratic penalties (H_B1) for {len(impossible_pairings)} impossible pairs across {len(tail_ids)} tails.")

Added quadratic penalties (H_B1) for 2 impossible pairs across 4 tails.


Convert to Ising Hamiltonian
This is the key step you asked about. We have defined our problem as a QuadraticProgram. 

Now, we'll use Qiskit's converters to:Convert the constrained QuadraticProgram into an unconstrained QUBO.
 This automatically adds the penalties ($\gamma H_A$).
 
 Convert the QUBO (using 0/1 variables) into an Ising Hamiltonian (using -1/+1 spins).Note: For simplicity, I'm only including $H_A$ for this first conversion. We would add the other constraints ($H_{B1}$, $H_{B2}$, etc.) in the same way as Cell 5 before running this.

In [ ]:
#  Cell 6: Convert Full Problem to Ising 

print("Full Quadratic Program summary (with all constraints):")
print(qp.prettyprint())
print("\n" + "="*30 + "\n")

# 1. Create a converter to turn constraints (H_A) into penalties
#    All other penalties (H_B1, B3, B4) are already in the objective
qp_to_qubo = QuadraticProgramToQubo(penalty=params['gamma'])
qubo = qp_to_qubo.convert(qp)

print("Full QUBO summary (all constraints & objective combined):")
# This QUBO will be much larger and denser
# print(qubo.prettyprint()) # Uncomment to see the full QUBO
print(f"QUBO has {len(qubo.objective.linear.to_dict())} linear terms")
print(f"QUBO has {len(qubo.objective.quadratic.to_dict())} quadratic terms")
print("\n" + "="*30 + "\n")


# 2. Create a converter to map the QUBO (0/1) to an Ising model (-1/+1)
# 2. Convert QUBO to Ising Hamiltonian using Qiskit's built-in method
ising_hamiltonian, ising_offset = qubo.to_ising()

print(f"Ising Hamiltonian (operator form):")
print(ising_hamiltonian)
print(f"\nIsing Offset (constant energy): {ising_offset}")

# This 'ising_hamiltonian' is the final operator to feed into QAOA

Full Quadratic Program summary (with all constraints):
Problem name: Tail-Assignment-Problem

Minimize
  8.5*q_F1_A101*q_F2_A101 + 8.5*q_F1_B202*q_F2_B202 + 8.5*q_F1_C303*q_F2_C303
  + 8.5*q_F1_D404*q_F2_D404 + 8.5*q_F3_A101*q_F5_A101 + 8.5*q_F3_B202*q_F5_B202
  + 8.5*q_F3_C303*q_F5_C303 + 8.5*q_F3_D404*q_F5_D404

Subject to
  Linear constraints (5)
    q_F1_A101 + q_F1_B202 + q_F1_C303 + q_F1_D404 == 1  'H_A_F1'
    q_F2_A101 + q_F2_B202 + q_F2_C303 + q_F2_D404 == 1  'H_A_F2'
    q_F3_A101 + q_F3_B202 + q_F3_C303 + q_F3_D404 == 1  'H_A_F3'
    q_F4_A101 + q_F4_B202 + q_F4_C303 + q_F4_D404 == 1  'H_A_F4'
    q_F5_A101 + q_F5_B202 + q_F5_C303 + q_F5_D404 == 1  'H_A_F5'

  Binary variables (20)
    q_F1_A101 q_F1_B202 q_F1_C303 q_F1_D404 q_F2_A101 q_F2_B202 q_F2_C303
    q_F2_D404 q_F3_A101 q_F3_B202 q_F3_C303 q_F3_D404 q_F4_A101 q_F4_B202
    q_F4_C303 q_F4_D404 q_F5_A101 q_F5_B202 q_F5_C303 q_F5_D404



Full QUBO summary (all constraints & objective combined):
QUBO has 20 linear terms


Set Up Solvers (Classical and Quantum)
First, we'll set up a classical solver (NumPyMinimumEigensolver) to find the exact correct answer. This gives us a baseline to check our quantum result against.

Then, we'll set up the QAOA algorithm. It needs:

A Sampler: This is the "backend" that runs the quantum circuit. We'll use a fast AerSimulator.

An Optimizer: This is the classical part that tunes the QAOA circuit's parameters. COBYLA is a good choice.

In [ ]:
#  Cell 7: Set Up Solvers (Classical and Quantum) 

#  1. Classical Baseline Solver 
# This solver finds the exact ground truth answer to the Ising problem.
numpy_solver = NumPyMinimumEigensolver()


#  2. Quantum QAOA Solver 

# a) Set up the classical optimizer that QAOA will use
# This optimizer finds the best angles (gamma, beta) for the circuit
optimizer = COBYLA(maxiter=200)

# b) Set up the quantum backend
# We'll use a local Aer simulator
# For a real quantum computer, you would change this.
sampler = AerSampler()

# c) Create the QAOA instance
qaoa = QAOA(
    sampler=sampler,
    optimizer=optimizer,
    reps=1  # 'p' value for QAOA. Start with 1 layer.
)

print("Classical and QAOA solvers initialized.")

Classical and QAOA solvers initialized.


Execute the Solvers
Now, we run both solvers on our ising_hamiltonian. This will take a moment, especially the QAOA part.

In [ ]:
#  Cell 7 (Revised): Set Up Solvers (Classical and VQE) 
from qiskit_algorithms import SamplingVQE
#  1. Classical Baseline Solver 
numpy_solver = NumPyMinimumEigensolver()


#  2. Quantum VQE Solver 

# a) Set up the classical optimizer
optimizer = COBYLA(maxiter=200)

# b) Set up the quantum primitive
# VQE needs an Estimator to find the energy (eigenvalue)
#estimator = Estimator()

# c) Manually create the QAOA 'ansatz' (circuit)
#    We use the Ising Hamiltonian from Cell 6
ansatz = QAOAAnsatz(cost_operator=ising_hamiltonian, reps=1)

# d) Create the VQE instance
#    VQE *does* accept an 'ansatz' argument.
vqe = SamplingVQE(
    sampler=sampler,
    ansatz=ansatz,
    optimizer=optimizer
)

print("Classical and VQE solvers initialized.")

Classical and VQE solvers initialized.


In [ ]:
# Cell 7 (QAOA Version): Set Up Optimizers 

#  1. Classical Baseline Solver 
numpy_solver = NumPyMinimumEigensolver()

#  2. Quantum Solver (SamplingVQE + QAOAAnsatz) 

# a) Set up the classical optimizer
optimizer = COBYLA(maxiter=500)  # High iterations

# b) Set up the quantum backend
sampler = AerSampler()

# c) Create the QAOA ansatz circuit
#    'ising_hamiltonian' 
ansatz = QAOAAnsatz(cost_operator=ising_hamiltonian, reps=3) # High reps

# d) Transpile the ansatz for the Aer simulator
aer_backend = AerSimulator()
transpiled_ansatz = transpile(ansatz, backend=aer_backend)

# e) Create the raw solver: SamplingVQE
#    (We use SamplingVQE as the "engine" for our QAOA ansatz)
sampling_vqe_qaoa = SamplingVQE(
    sampler=sampler,
    optimizer=optimizer,
    ansatz=transpiled_ansatz
)

# 3. Create the High-Level Optimizers 
# Wrap the raw solvers in the MinimumEigenOptimizer
numpy_optimizer = MinimumEigenOptimizer(numpy_solver)
qaoa_optimizer = MinimumEigenOptimizer(sampling_vqe_qaoa)


print(f"Classical and QAOA Optimizers initialized for {ansatz.num_qubits} qubits.")

Classical and QAOA Optimizers initialized for 20 qubits.


In [ ]:
#  Cell 8: Execute the Optimizers 

print("--- Running Classical Optimizer (Baseline) ---")
# We pass the QUBO (from Cell 6) to the .solve() method
classical_result = numpy_optimizer.solve(qubo)
print("Classical optimizer finished.")


print("\n--- Running QAOA Optimizer ---")
# We pass the same QUBO to the QAOA-based optimizer
qaoa_result = qaoa_optimizer.solve(qubo)
print("QAOA optimizer finished.")


print("\n--- Results (Final Cost) ---")
# .fval is the final function value (cost). No offset needed.
print(f"Classical (Exact) Cost: {classical_result.fval:.4f}")
print(f"QAOA (Quantum) Cost:    {qaoa_result.fval:.4f}")

--- Running Classical Optimizer (Baseline) ---
Classical optimizer finished.

--- Running QAOA Optimizer ---
Classical optimizer finished.

--- Running QAOA Optimizer ---
QAOA optimizer finished.

--- Results (Final Cost) ---
Classical (Exact) Cost: 0.0000
QAOA (Quantum) Cost:    0.0000
QAOA optimizer finished.

--- Results (Final Cost) ---
Classical (Exact) Cost: 0.0000
QAOA (Quantum) Cost:    0.0000


In [ ]:
#  Cell 9: Interpret the Solution 

def get_solution_assignments(solution, qubo): # 'solution' is an OptimizationResult
    """Parses an OptimizationResult and maps it to flight assignments."""
    
    # .x is the binary string (e.g., [0, 1, 0, 0, ...])
    solution_bits = solution.x
    
    # Create a reverse map from variable index to name
    var_names = [var.name for var in qubo.variables]
    
    assignments = {}
    total_cost = 0
    
    print("\n--- Optimal Schedule ---")
    for i, bit in enumerate(solution_bits):
        if bit == 1:
            var_name = var_names[i]
            # var_name is "q_{f}_{t}", so split it
            _, f_id, t_id = var_name.split('_')
            
            print(f"Assign Flight {f_id} -> Tail {t_id}")
            assignments[f_id] = t_id

    return assignments

#  Use the helper function 
# Pass the results from Cell 8
print("Parsing Classical (Exact) Solution:")
classical_assignments = get_solution_assignments(classical_result, qubo)

print("\nParsing QAOA Solution:")
qaoa_assignments = get_solution_assignments(qaoa_result, qubo)

Parsing Classical (Exact) Solution:

--- Optimal Schedule ---
Assign Flight F1 -> Tail B202
Assign Flight F2 -> Tail A101
Assign Flight F3 -> Tail B202
Assign Flight F4 -> Tail A101
Assign Flight F5 -> Tail A101

Parsing QAOA Solution:

--- Optimal Schedule ---
Assign Flight F1 -> Tail A101
Assign Flight F2 -> Tail D404
Assign Flight F3 -> Tail C303
Assign Flight F4 -> Tail D404
Assign Flight F5 -> Tail A101


In [ ]:
#  Cell 7 (VQE Version - Revised): Set Up Optimizers 

#  1. Classical Baseline Solver 
numpy_solver = NumPyMinimumEigensolver()

#  2. Quantum Solver (SamplingVQE + RealAmplitudes Ansatz) 
optimizer = COBYLA(maxiter=500)
sampler = AerSampler()
num_qubits = ising_hamiltonian.num_qubits
ansatz = RealAmplitudes(num_qubits, entanglement='linear', reps=3)
aer_backend = AerSimulator()
transpiled_ansatz = transpile(ansatz, backend=aer_backend)

# Create the raw VQE solver instance
sampling_vqe = SamplingVQE(
    sampler=sampler,
    optimizer=optimizer,
    ansatz=transpiled_ansatz
)

#  3. Create the High-Level Optimizers 
# Wrap the raw solvers in the MinimumEigenOptimizer
# This is the class that knows how to translate the results back to binary
numpy_optimizer = MinimumEigenOptimizer(numpy_solver)
vqe_optimizer = MinimumEigenOptimizer(sampling_vqe)

print(f"Classical and VQE Optimizers initialized for {num_qubits} qubits.")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_22388\4053735373.py:10: DeprecationWarning: The class ``qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.real_amplitudes instead.
  ansatz = RealAmplitudes(num_qubits, entanglement='linear', reps=3)


Classical and VQE Optimizers initialized for 20 qubits.


In [ ]:
#  Cell 8: Execute the Optimizers 

print("--- Running Classical Optimizer (Baseline) ---")
# We pass the QUBO (from Cell 6) to the .solve() method
classical_result = numpy_optimizer.solve(qubo)
print("Classical optimizer finished.")


print("\n--- Running VQE Optimizer ---")
# We pass the same QUBO to the VQE-based optimizer
vqe_result = vqe_optimizer.solve(qubo)
print("VQE optimizer finished.")


print("\n--- Results (Final Cost) ---")
# .fval is the final function value (cost). No offset needed.
print(f"Classical (Exact) Cost: {classical_result.fval:.4f}")
print(f"VQE (Quantum) Cost:     {vqe_result.fval:.4f}")

--- Running Classical Optimizer (Baseline) ---
Classical optimizer finished.

--- Running VQE Optimizer ---
Classical optimizer finished.

--- Running VQE Optimizer ---
VQE optimizer finished.

--- Results (Final Cost) ---
Classical (Exact) Cost: 0.0000
VQE (Quantum) Cost:     0.0000
VQE optimizer finished.

--- Results (Final Cost) ---
Classical (Exact) Cost: 0.0000
VQE (Quantum) Cost:     0.0000


In [ ]:
# Cell 9: Interpret the Solution

def get_solution_assignments(solution, qubo): # Renamed 'result' to 'solution'
    """Parses an OptimizationResult and maps it to flight assignments."""
    
    # .x is the binary string (e.g., [0, 1, 0, 0, ...])
    solution_bits = solution.x  # <-- THE FIX
    
    # Create a reverse map from variable index to name
    var_names = [var.name for var in qubo.variables]
    
    assignments = {}
    total_cost = 0
    
    print("\n--- Optimal Schedule ---")
    for i, bit in enumerate(solution_bits):
        if bit == 1:
            var_name = var_names[i]
            # var_name is "q_{f}_{t}", so split it
            _, f_id, t_id = var_name.split('_')
            
            print(f"Assign Flight {f_id} -> Tail {t_id}")
            assignments[f_id] = t_id

    return assignments

#  Use the helper function 
# Now we pass the correct OptimizationResult objects
print("Parsing Classical (Exact) Solution:")
classical_assignments = get_solution_assignments(classical_result, qubo)

print("\nParsing VQE Solution:")
vqe_assignments = get_solution_assignments(vqe_result, qubo)

Parsing Classical (Exact) Solution:

--- Optimal Schedule ---
Assign Flight F1 -> Tail B202
Assign Flight F2 -> Tail A101
Assign Flight F3 -> Tail B202
Assign Flight F4 -> Tail A101
Assign Flight F5 -> Tail A101

Parsing VQE Solution:

--- Optimal Schedule ---
Assign Flight F1 -> Tail A101
Assign Flight F2 -> Tail B202
Assign Flight F3 -> Tail C303
Assign Flight F4 -> Tail B202
Assign Flight F5 -> Tail D404
